<a href="https://colab.research.google.com/github/OsvaldoMS1982/TFM-NLP2SQL/blob/Fase-2/Fase2_EntrenamientodeModelos_BART_LARGE_NL2SQL_FineTuned.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Montar Drive

In [ ]:
from google.colab import drive
import os

# Montar Google Drive
drive.mount('/content/drive')

# Definir la ruta del dataset Spider en Google Drive
SPIDER_PATH = "/content/drive/My Drive/spider"

# Verificar que los archivos están en la ubicación correcta
print("Archivos en Spider:", os.listdir(SPIDER_PATH))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Archivos en Spider: ['train_gold.sql', 'dev_gold.sql', 'dev.json', 'train_others.json', 'train_spider.json', 'tables.json', 'README.txt', 'test_tables.json', 'test.json', 'test_gold.sql', '.DS_Store', 'test_database', 'database', 'train_spider_fixed.json', 'database.sqlite']


Cargar los Datos de Spider

In [ ]:
import json

# Cargar datos de entrenamiento y validación
with open(f"{SPIDER_PATH}/train_spider.json", "r") as f:
    train_data = json.load(f)

with open(f"{SPIDER_PATH}/dev.json", "r") as f:
    val_data = json.load(f)

print("Ejemplo de entrenamiento:", train_data[0])
print("Ejemplo de validación:", val_data[0])


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Preprocesar los Datos para los Modelos

In [ ]:
import pandas as pd

# Función para convertir Spider a formato de entrenamiento
def preprocess_spider(data):
    inputs = []
    targets = []

    for item in data:
        question = item["question"]  # Pregunta en lenguaje natural
        sql_query = item["query"]  # SQL correspondiente

        # Formato de entrada para los modelos
        inputs.append(f"Translate to SQL: {question}")
        targets.append(sql_query)

    return pd.DataFrame({"input": inputs, "target": targets})

# Convertir datos de entrenamiento y validación
train_df = preprocess_spider(train_data)
val_df = preprocess_spider(val_data)

print("Ejemplo de entrada para T5:", train_df.head())


Ejemplo de entrada para T5:                                                input  \
0  Translate to SQL: How many heads of the depart...   
1  Translate to SQL: List the name, born state an...   
2  Translate to SQL: List the creation year, name...   
3  Translate to SQL: What are the maximum and min...   
4  Translate to SQL: What is the average number o...   

                                              target  
0         SELECT count(*) FROM head WHERE age  >  56  
1  SELECT name ,  born_state ,  age FROM head ORD...  
2  SELECT creation ,  name ,  budget_in_billions ...  
3  SELECT max(budget_in_billions) ,  min(budget_i...  
4  SELECT avg(num_employees) FROM department WHER...  


In [ ]:
pip install --upgrade huggingface_hub transformers

Función para Tokenización

In [ ]:

from transformers import AutoTokenizer

def tokenize_data(df, model_name):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    inputs = tokenizer(df["input"].tolist(), padding="max_length", truncation=True, return_tensors="pt", max_length=512)
    targets = tokenizer(df["target"].tolist(), padding="max_length", truncation=True, return_tensors="pt", max_length=128)
    return inputs, targets, tokenizer


Crear Dataset PyTorch

In [ ]:
import torch
from torch.utils.data import Dataset

class SpiderDataset(Dataset):
    def __init__(self, inputs, targets):
        self.inputs = inputs
        self.targets = targets

    def __len__(self):
        return len(self.inputs["input_ids"])

    def __getitem__(self, idx):
        return {
            "input_ids": self.inputs["input_ids"][idx],
            "attention_mask": self.inputs["attention_mask"][idx],
            "labels": self.targets["input_ids"][idx]
        }


Configurar y Entrenar el modelo BART-LARGE-NL2SQL

In [ ]:
from transformers import AutoModelForSeq2SeqLM, Trainer, TrainingArguments
from transformers import EarlyStoppingCallback


# Modelos en Hugging Face
models = {
    "BART-LARGE-NL2SQL": "SwastikM/bart-large-nl2sql",
    #"BART-LARGE-NL2SQL": "SwastikM/bart-large-nl2sql",
    #"NL2SQL-StarCoder-15B": "gabrielpondc/NL2SQL-StarCoder-15B",
  #  "AutoSQL-nl2sql-8b": "xbrain/AutoSQL-nl2sql-1.0-8b",
   # "T5-Small-NL2SQL": "Shritama/t5-small-finetuned-nl2sql"
}

# Entrenar cada modelo
for model_name, model_path in models.items():
    print(f"\n🔵 Entrenando {model_name}...\n")

    # Tokenizar datos
    train_inputs, train_targets, tokenizer = tokenize_data(train_df, model_path)
    val_inputs, val_targets, _ = tokenize_data(val_df, model_path)

    # Crear dataset
    train_dataset = SpiderDataset(train_inputs, train_targets)
    val_dataset = SpiderDataset(val_inputs, val_targets)

    # Cargar modelo
    model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

    # Configurar entrenamiento
    training_args = TrainingArguments(
        #Fine Tune: se agrega fp16=True para reducir memoria y mejorar el entrenamiento
        fp16=True,
        output_dir=f"./{model_name}_spider",
        evaluation_strategy="epoch",
        #Fine Tune: reducimos el training batch y eval batch de 8 a 4 para mejorar cuidar el consumo de memoria
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        #Fine Tune reducimos el learning_rate=3e-5,para hacerlo mas estable
        learning_rate=3e-5,
        weight_decay=0.01,
        load_best_model_at_end=True, #Requerido para Early Stopping
        #Aumentamos cantidad de epocas a 5
        num_train_epochs=5,
        logging_dir=f"./logs/{model_name}",
        save_strategy="epoch"
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        #optimizers=(trainer.optimizer, lr_scheduler)
    )

    trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=2))


    # Iniciar entrenamiento
    trainer.train()

    # Guardar modelo en Google Drive
    save_path = f"/content/drive/My Drive/spider_models_fine_Tuned/{model_name}"
    model.save_pretrained(save_path)
    tokenizer.save_pretrained(save_path)
    print(f"✅ Modelo {model_name} guardado en {save_path}")



🔵 Entrenando BART-LARGE-NL2SQL...



/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.096600,0.320704
2,0.044300,0.353610


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:2758: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


Probar el Modelo Entrenado

In [ ]:
!pip install sacrebleu datasets torch tabulate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 94.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 68.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch

# 📌 Lista de modelos a evaluar
models_to_test = {

    "BART-LARGE-NL2SQL": "/content/drive/MyDrive/spider_models_fine_Tuned/BART-LARGE-NL2SQL",

}

# 📌 Diccionario para almacenar modelos y tokenizers
models_dict = {}

for model_name, model_path in models_to_test.items():
    try:
        model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
        tokenizer = AutoTokenizer.from_pretrained(model_path)
        models_dict[model_name] = {"model": model, "tokenizer": tokenizer}
        print(f"✅ {model_name} cargado correctamente.")
    except Exception as e:
        print(f"❌ Error al cargar {model_name}: {e}")


/usr/local/lib/python3.11/dist-packages/transformers/models/bart/configuration_bart.py:176: UserWarning: Please make sure the config includes `forced_bos_token_id=0` in future versions. The config can simply be saved and uploaded again to be fixed.
  warnings.warn(


✅ BART-LARGE-NL2SQL cargado correctamente.


In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model_name = "BART-LARGE-NL2SQL"
model_path = f"/content/drive/My Drive/spider_models_fine_Tuned/{model_name}"

try:
    model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    print("Modelo y tokenizer cargados exitosamente.")
except Exception as e:
    print(f"Error al cargar el modelo/tokenizer: {e}")


Modelo y tokenizer cargados exitosamente.


In [ ]:
#print("Generando SQL...")  # Esto verificará si se llega a esta línea
#print("Modelo cargado:", model)

def generate_sql(question):
    input_text = f"Translate to SQL: {question}"
    inputs = tokenizer(input_text, return_tensors="pt")

    print("Texto de entrada:", input_text)  # Verificar lo que se está enviando al modelo

    # Generar SQL
    output = model.generate(**inputs, max_length=50, do_sample=True)

    if len(output) == 0:
        return "Error: El modelo no generó salida."

    sql_query = tokenizer.decode(output[0], skip_special_tokens=True)

    return sql_query



resultado = generate_sql("total employees")
print(resultado)  # Esto imprimirá la consulta generada o el mensaje de error

Texto de entrada: Translate to SQL: total employees


Both `max_new_tokens` (=128) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


SELECT Total_employees FROM employee


In [ ]:
question = "count of total employees?"
input_text = f"Translate to SQL: {question}"
inputs = tokenizer(input_text, return_tensors="pt")

print("Tokens generados:", inputs)


Tokens generados: {'input_ids': tensor([[    0, 19163, 19593,     7, 41614,    35,  3212,     9,   746,  1321,
           116,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [ ]:
output = model.generate(**inputs)
print("Salida cruda del modelo:", output)

Salida cruda del modelo: tensor([[2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 2]])


In [ ]:
#question = "How many employees are in the company?"
#print("Modelo cargado:", model)


def generate_sql(question):
    input_text = f"Translate to SQL: {question}"
    inputs = tokenizer(input_text, return_tensors="pt")
    print("Texto de entrada:", input_text)  # Verificar lo que se está enviando al modelo
    # Parámetros de generación ajustados
    output = model.generate(**inputs, max_length=50, do_sample=True)

    if len(output) == 0:
        return "Error: El modelo no generó salida."

    sql_query = tokenizer.decode(output[0], skip_special_tokens=True)
    return sql_query


resultado = generate_sql("Total Employees?")
print(resultado)  # Esto imprimirá la consulta generada o el mensaje de error


Texto de entrada: Translate to SQL: Total Employees?


Both `max_new_tokens` (=128) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


SELECT Total_Employees FROM Employees


In [ ]:
# Prueba rápida en validación
sample = val_df.iloc[0]["input"]
inputs = tokenizer(sample, return_tensors="pt")
output = model.generate(**inputs)

print("Pregunta de validación:", sample)
print("SQL Generado:", tokenizer.decode(output[0], skip_special_tokens=True))


Pregunta de validación: Translate to SQL: How many singers do we have?
SQL Generado: 


Probar el Modelo

In [ ]:

def generate_sql(question):
    input_text = f"Translate to SQL: {question}"
    inputs = tokenizer(input_text, return_tensors="pt")
    print("Texto de entrada:", input_text)  # Verificar lo que se está enviando al modelo
    # Parámetros de generación ajustados
    output = model.generate(**inputs, max_length=50, do_sample=True)

    if len(output) == 0:
        return "Error: El modelo no generó salida."

    sql_query = tokenizer.decode(output[0], skip_special_tokens=True)
    return sql_query


resultado = generate_sql("Total Employees?")
print(resultado)  # Esto imprimirá la consulta generada o el mensaje de error


Texto de entrada: Translate to SQL: Total Employees?


Both `max_new_tokens` (=128) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


SELECT Total_of_employees FROM Employees


Generar Predicciones SQL para Validación

In [ ]:
import json

SPIDER_PATH = "/content/drive/My Drive/spider"

# 📌 Cargar datos de validación de Spider
with open(f"{SPIDER_PATH}/dev.json", "r") as f:
    val_data = json.load(f)

# 📌 Extraer preguntas y sus SQL correctos
questions = [item["question"] for item in val_data]
true_sql = [item["query"] for item in val_data]

# 📌 Función para generar SQL con cada modelo
def generate_sql(model, tokenizer, question):
    input_text = f"Translate to SQL: {question}"
    inputs = tokenizer(input_text, return_tensors="pt")

    output = model.generate(
        **inputs, max_length=128, temperature=0.7, top_p=0.9, num_return_sequences=1, repetition_penalty=1.2
    )
    return tokenizer.decode(output[0], skip_special_tokens=True)

# 📌 Generar consultas SQL con cada modelo
predictions = {}
for model_name, data in models_dict.items():
    print(f"\n🚀 Generando SQL con {model_name}...\n")
    model = data["model"]
    tokenizer = data["tokenizer"]
    predictions[model_name] = [generate_sql(model, tokenizer, q) for q in questions]



🚀 Generando SQL con BART-LARGE-NL2SQL...



/usr/local/lib/python3.11/dist-packages/transformers/generation/utils.py:1528: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed in v5. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or

Evaluar Modelo

In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.1 MB/s eta 0:00:00


In [ ]:
import evaluate
import sqlite3
from tabulate import tabulate

# 📌 Cargar métricas BLEU
bleu_metric = evaluate.load("sacrebleu")

# 📌 Función para calcular Exact Match
def exact_match(pred, true):
    return int(pred.strip().lower() == true.strip().lower())

# 📌 Función para ejecutar consultas SQL en SQLite
def execute_sql(query, conn):
    try:
        cursor = conn.cursor()
        cursor.execute(query)
        return cursor.fetchall()
    except Exception as e:
        return None  # Si la consulta es inválida, devuelve None

# 📌 Función para calcular Execution Accuracy
def execution_accuracy(predictions, references, conn):
    correct = sum(1 for pred, ref in zip(predictions, references) if execute_sql(pred, conn) == execute_sql(ref, conn))
    return (correct / len(references)) * 100

# 📌 Crear una base de datos en memoria para evaluar Execution Accuracy
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

# 🚀 Simulación de una base de datos (Asegúrate de usar una DB real en producción)
cursor.execute("CREATE TABLE employees (id INTEGER, name TEXT, salary INTEGER)")
cursor.executemany("INSERT INTO employees (id, name, salary) VALUES (?, ?, ?)", [
    (1, "Alice", 50000),
    (2, "Bob", 60000),
    (3, "Charlie", 70000)
])
conn.commit()

# 📌 Evaluar cada modelo
results = []

for model_name in models_dict.keys():
    pred_sql = predictions[model_name]

    # 📌 Calcular BLEU Score
    bleu_score = bleu_metric.compute(predictions=pred_sql, references=[[x] for x in true_sql])["score"]

    # 📌 Calcular Exact Match (EM)
    em_score = sum(exact_match(pred_sql[i], true_sql[i]) for i in range(len(true_sql))) / len(true_sql)

    # 📌 Calcular Execution Accuracy
    exec_acc = execution_accuracy(pred_sql, true_sql, conn)

    results.append([model_name, round(bleu_score, 2), round(em_score * 100, 2), round(exec_acc, 2)])

# 📌 Mostrar Resultados en Tabla
print("\n📊 **Comparación de Métricas**\n")
print(tabulate(results, headers=["Modelo", "BLEU Score", "Exact Match (%)", "Execution Accuracy (%)"], tablefmt="grid"))

# 📌 Cerrar la conexión con la base de datos
conn.close()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



📊 **Comparación de Métricas**

+-------------------+--------------+-------------------+--------------------------+
| Modelo            |   BLEU Score |   Exact Match (%) |   Execution Accuracy (%) |
+===================+==============+===================+==========================+
| BART-LARGE-NL2SQL |        32.47 |              3.09 |                    97.58 |
+-------------------+--------------+-------------------+--------------------------+


BLEU Score: 32.47 significa que el modelo produce SQL que tiene cierta similitud con la consulta esperada, pero no es una coincidencia exacta.

Exact Match: 3.09% es bajo → Significa que muy pocas consultas generadas coinciden exactamente con la esperada.

Execution Accuracy: 97.5% es excelente → Significa que, aunque el SQL generado no sea idéntico (según Exact Match), casi siempre obtiene los mismos resultados cuando se ejecuta.